<a href="https://colab.research.google.com/github/ajaysaikiran2208/Recommendation-Systems/blob/main/Drug_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##***K Nearest Neighbor recommendation system*** for Drug Recommendation 

K Nearest Neighbour finds the point depending upon the distance.we have,

***1.Euclidean Distance*** 

***2.Manhattan Distance***

In the Case of **Classification** the new point will be having the value of class which has **highest points** in them.For example if we are classifying Male and Female with K=5[It should be always an odd value] and in this case 3 values are male and 2 values are female then new point will be male.

In the case of **Regression** the new point will be having the value of the **mean** of all the points since this is a continous data

***Outliers*** and ***Imbalance*** data will impact the performance of the model.

##Creating Recommendation Systems using Nearest Neighbors

Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score,accuracy_score
import pickle

#Reading DATASET

In [11]:
drugs=pd.read_csv("/content/drugsComTrain_raw.csv")
drugs.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,03-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


#Separating Reviews data for Sentiment analysis

In [12]:
reviews = pd.DataFrame(drugs, columns = ['review','drugName','condition'])
reviews.head()

,review,drugName,condition
0,"""It has no side effect, I take it in combinati...",Valsartan,Left Ventricular Dysfunction
1,"""My son is halfway through his fourth week of ...",Guanfacine,ADHD
2,"""I used to take another oral contraceptive, wh...",Lybrel,Birth Control
3,"""This is my first time using any form of birth...",Ortho Evra,Birth Control
4,"""Suboxone has completely turned my life around...",Buprenorphine / naloxone,Opiate Dependence


Grouping on drug name and total rating count

In [18]:
combine_drug_rating=drugs.dropna(axis=0,subset=["drugName"])
drug_ratingcount=(combine_drug_rating.groupby(by=['drugName'])['condition']).count().reset_index().rename(columns={'condition':'totalratingcount'})[['drugName','totalratingcount']]
drug_ratingcount.head()

SyntaxError: ignored

In [5]:
drug_ratingcount=combine_drug_rating.merge(drug_ratingcount,left_on='drugName',right_on='drugName',how='left')
drug_ratingcount.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,totalratingcount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,82
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,170
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,14
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,03-Nov-15,10,290
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,599


Statistical data 

In [10]:
pd.set_option('display.float_format',lambda x:'%.3f'%x)
print(drug_ratingcount['drugName'].describe())

count             214949
unique              3671
top       Levonorgestrel
freq                4925
Name: drugName, dtype: object


Filtering data by setting a threshold value

In [7]:
totalratingcount_threshold=50
rating_popular_drug=drug_ratingcount.query('totalratingcount>=@totalratingcount_threshold')
rating_popular_drug.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,totalratingcount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,82
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,170
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,03-Nov-15,10,290
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,599
5,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,368


In [19]:
drug_ratingcount.shape

(3671, 2)

In [20]:
final_dataset = drugs.pivot_table(index='drugName',columns='condition',values='rating').fillna(0)
final_dataset.head()

condition,0</span> users found this comment helpful.,100</span> users found this comment helpful.,105</span> users found this comment helpful.,10</span> users found this comment helpful.,110</span> users found this comment helpful.,11</span> users found this comment helpful.,121</span> users found this comment helpful.,123</span> users found this comment helpful.,12</span> users found this comment helpful.,135</span> users found this comment helpful.,13</span> users found this comment helpful.,142</span> users found this comment helpful.,145</span> users found this comment helpful.,146</span> users found this comment helpful.,14</span> users found this comment helpful.,15</span> users found this comment helpful.,16</span> users found this comment helpful.,17</span> users found this comment helpful.,18</span> users found this comment helpful.,19</span> users found this comment helpful.,1</span> users found this comment helpful.,20</span> users found this comment helpful.,21</span> users found this comment helpful.,22</span> users found this comment helpful.,23</span> users found this comment helpful.,24</span> users found this comment helpful.,25</span> users found this comment helpful.,26</span> users found this comment helpful.,27</span> users found this comment helpful.,28</span> users found this comment helpful.,29</span> users found this comment helpful.,2</span> users found this comment helpful.,30</span> users found this comment helpful.,31</span> users found this comment helpful.,32</span> users found this comment helpful.,33</span> users found this comment helpful.,34</span> users found this comment helpful.,35</span> users found this comment helpful.,36</span> users found this comment helpful.,37</span> users found this comment helpful.,...,Yellow Fever Prophylaxis,Zollinger-Ellison Syndrome,acial Lipoatrophy,acial Wrinkles,actor IX Deficiency,ailure to Thrive,amilial Cold Autoinflammatory Syndrome,amilial Mediterranean Feve,atigue,cal Segmental Glomerulosclerosis,emale Infertility,eve,ge (amlodipine / valsartan),ge HCT (amlodipine / hydrochlorothiazide / valsartan),ibrocystic Breast Disease,ibromyalgia,lic Acid Deficiency,llicle Stimulation,llicular Lymphoma,m Pain Disorde,mance Anxiety,me,min / pioglitazone),min / rosiglitazone),min / saxagliptin),min / sitagliptin),min),mis,mist (,moterol / mometasone),moterol),mulation) (phenylephrine),t Care,t Pac with Cyclobenzaprine (cyclobenzaprine),tic (mycophenolic acid),unctional Gastric Disorde,ungal Infection Prophylaxis,ungal Pneumonia,von Willebrand's Disease,zen Shoulde
drugName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A + D Cracked Skin Relief,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A / B Otic,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Abacavir / dolutegravir / lamivudine,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.

In [21]:
from scipy.sparse import csr_matrix
drug_features_matrix=csr_matrix(final_dataset.values)
from sklearn.neighbors import NearestNeighbors
model_knn=NearestNeighbors(metric="cosine",algorithm='brute')
model_knn.fit(drug_features_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [22]:
final_dataset.shape

(3667, 916)

In [29]:
query_index=np.random.choice(final_dataset.shape[1])
print(query_index)
distances,indices=model_knn.kneighbors(final_dataset.iloc[query_index,:].values.reshape(1,-1),n_neighbors=10)

676


In [25]:
final_dataset.head()

condition,0</span> users found this comment helpful.,100</span> users found this comment helpful.,105</span> users found this comment helpful.,10</span> users found this comment helpful.,110</span> users found this comment helpful.,11</span> users found this comment helpful.,121</span> users found this comment helpful.,123</span> users found this comment helpful.,12</span> users found this comment helpful.,135</span> users found this comment helpful.,13</span> users found this comment helpful.,142</span> users found this comment helpful.,145</span> users found this comment helpful.,146</span> users found this comment helpful.,14</span> users found this comment helpful.,15</span> users found this comment helpful.,16</span> users found this comment helpful.,17</span> users found this comment helpful.,18</span> users found this comment helpful.,19</span> users found this comment helpful.,1</span> users found this comment helpful.,20</span> users found this comment helpful.,21</span> users found this comment helpful.,22</span> users found this comment helpful.,23</span> users found this comment helpful.,24</span> users found this comment helpful.,25</span> users found this comment helpful.,26</span> users found this comment helpful.,27</span> users found this comment helpful.,28</span> users found this comment helpful.,29</span> users found this comment helpful.,2</span> users found this comment helpful.,30</span> users found this comment helpful.,31</span> users found this comment helpful.,32</span> users found this comment helpful.,33</span> users found this comment helpful.,34</span> users found this comment helpful.,35</span> users found this comment helpful.,36</span> users found this comment helpful.,37</span> users found this comment helpful.,...,Yellow Fever Prophylaxis,Zollinger-Ellison Syndrome,acial Lipoatrophy,acial Wrinkles,actor IX Deficiency,ailure to Thrive,amilial Cold Autoinflammatory Syndrome,amilial Mediterranean Feve,atigue,cal Segmental Glomerulosclerosis,emale Infertility,eve,ge (amlodipine / valsartan),ge HCT (amlodipine / hydrochlorothiazide / valsartan),ibrocystic Breast Disease,ibromyalgia,lic Acid Deficiency,llicle Stimulation,llicular Lymphoma,m Pain Disorde,mance Anxiety,me,min / pioglitazone),min / rosiglitazone),min / saxagliptin),min / sitagliptin),min),mis,mist (,moterol / mometasone),moterol),mulation) (phenylephrine),t Care,t Pac with Cyclobenzaprine (cyclobenzaprine),tic (mycophenolic acid),unctional Gastric Disorde,ungal Infection Prophylaxis,ungal Pneumonia,von Willebrand's Disease,zen Shoulde
drugName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A + D Cracked Skin Relief,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A / B Otic,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Abacavir / dolutegravir / lamivudine,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.

In [30]:
for i in range(0,len(distances.flatten())):
  if i ==0:
    print('Recommendations for {0}:\n'.format(final_dataset.index[query_index]))
  else:
    print('{0}:{1},with distance of {2}:'.format(i,final_dataset.index[indices.flatten()[i]],distances.flatten()[i]))

Recommendations for Cheratussin DAC:

1:Ed A-Hist,with distance of 0.0:
2:Robitussin Cold Cough and Flu,with distance of 0.0:
3:Ephedrine / guaifenesin,with distance of 0.0:
4:Acetaminophen / chlorpheniramine / dextromethorphan / pseudoephedrine,with distance of 0.0:
5:Sudafed PE Severe Cold,with distance of 0.0:
6:Acetaminophen / dextromethorphan / doxylamine / pseudoephedrine,with distance of 0.0:
7:Acetaminophen / dextromethorphan / guaifenesin / pseudoephedrine,with distance of 0.0:
8:Codeine / guaifenesin / pseudoephedrine,with distance of 0.0:
9:Acetaminophen / diphenhydramine / phenylephrine,with distance of 0.0:
